## Travel Radius Calculator

### Callum Staff, September 2017


This script is for running bulk calls of the freemaptools.com's feature for [calculating how far you can travel in any direction from a particular point](https://www.freemaptools.com/how-far-can-i-travel.htm).

It uses the Selenium webdriver to input the data to the website and then download the corresponding kml file which is output. This can then be visualised, although code for this is not provided currently.

The data required for a particular point is as follows:
* The name of the place in question, for use in the filename
* The postcode of the place in question
* Whether the location is predominantly in an urban or rural location

### Key Function Features

* Measurements are in miles (this can be changed in the function below)
* A medium accuracy radius is calculated - more code would need to be added into the function to change this to low or high accuracy
* The method of transport is driving - more code would need to be added into the function to change this
* For areas assumed to be close to faster, rural roads, a speed of 49mph is assumed - this is the average speed on national speed limit roads, as detailed in [these Department for Transport (DfT) statistics](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/623261/vehicle-speed-compliance-statistics-2016.pdf)
* For areas assumed to be in more urban areas, a speed of 37mph is assumed. This is based in an average speed on urban roads of 25mph (also from the above statistics) and an average speed of 49mph on national speed limit roads, with exactly half the time spent at each speed. This assumption was built from a travel time of one hour, but might have to be adjusted if the travel time is changed
* The function will wait for 120 seconds for the radius to be calculated before timing out, this may need to be increased or decreased, particularly if the accuracy of the radius is altered

In [ ]:
#Load required libraries
import os #A module to carry out operation system operations
import unicodedata #A module to encode data in unicode
from selenium import webdriver #The Selenium webdriver and associated functions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time #A module to pause code and measure time
import pandas as pd #A module for handling dataframes

In [ ]:
#Read in required data

In [ ]:
#Create function
def travel_radius(postcode,rurality,name,lengthoftime):
    
    #Specify path where webdriver is located
    chrome_driver_path = "../path/to/webdriver"

    #Create a new Chrome session
    driver = webdriver.Chrome(chrome_driver_path)
    driver.implicitly_wait(30)
    driver.maximize_window()

    #Navigate to the URL
    driver.get("https://www.freemaptools.com/how-far-can-i-travel.htm")
    
    #Input location
    location = driver.find_element_by_id("goto")
    location.send_keys(postcode)
    
    #Set measurement unit to miles
    driver.find_element_by_xpath("//td/input[@type='radio' and @value='miles']").click()
    
    #Set speed dependent on urban or rural location
    speed = driver.find_element_by_id("speedslider")
    if rurality == "Urban":
        speed.send_keys("37")
    elif rurality == "Rural":
        speed.send_keys("49")
    
    #Specify travel time
    timing = driver.find_element_by_id("timex")
    timing.send_keys(lengthoftime)
    
    #Click button to calculate radius
    driver.find_element_by_xpath("//p[@class = 'fmtbutton']").click()
    
    #When radius has been calculated, download the kml file
    try:
        element = WebDriverWait(driver, 120).until(
            EC.text_to_be_present_in_element((By.XPATH, "//div[@id='wait']/p"),"Finished!")
        )
    finally:
        #For Chrome drivers, the window needs to be expanded to ensure the driver can find the download kml button
        driver.set_window_size(1800, 600)
        driver.maximize_window()
        driver.find_element_by_xpath("//input[@id = 'btn_genkml']").click()
        #Pause 10 seconds to allow the file to be downloaded
        time.sleep(10)
        #Move file from downloads to specific folder
        for file in os.listdir("/path/to/Downloads"):
            if file.endswith(".kml"):
                os.rename("/path/to/Downloads/"+file, "/path/to/new/folder/"+name+".kml")
        #Close the driver browser page
        driver.quit()

In [ ]:
#Apply over dataframe
data.apply(lambda x: travel_radius(x['Postcode'],x['Urban_Rural'],x['Location_Name']), axis=1)